# Compare the skip reasons between main and branch 

Input to this notebooks are the local logs generated by running something like 
```
python /mnt/remote/data/benjin/test_steps_utils/test_step.py --step_id targetless_extrinsics --test_type run-local --make_path ~/Development/wcs/WayveCode/wayve/services/data/lakehouse/jobs/slam --num_run_ids 20 --out_file /mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/run_local_main.log
``` 

and the cloud logs which is the csv generated from datadog.

In [56]:
from wayve.services.data.lakehouse.common.external_access.wayve_delta_table import WayveDeltaTable
from pathlib import Path
import pandas as pd

In [ ]:
RUN_ID_STR = """
oya/2022-04-05--13-18-39--gamepad-control,369588cf-9afb-401b-8d81-0c7e6f1102c4/2021-02-15--17-27-00--c1a97404-b6b0-4d56-a227-561f3a55cbb8,brizo/2021-07-20--10-20-56--Aysen,369588cf-9afb-401b-8d81-0c7e6f1102c4/2021-10-06--14-46-24--bde9c3f8-c655-4b2d-9097-2ab503264458,oya/2022-07-21--12-33-03--gamepad-control,e3969b44-315b-45b0-9038-a229c2a95e79/2022-04-20--10-57-51--a0af32fa-4513-4110-9265-e3ed9bfd0b09,884d2dba-f420-46f9-a7d6-49bb2fd6e23b/2022-01-21--09-47-03--4e185ba0-028e-4419-8509-94c6bb93d540,sedna/2023-03-15--12-05-00--Danny-Ly,2edd5814-5a9e-417b-9493-8f4430fb9af2/2021-11-17--09-49-36--a566582b-b612-4f8a-8039-39007ffcc714,16c8b7bb-f6ea-4528-aae1-697974cb547a/2021-02-24--15-53-25--d5291238-34f7-4f22-a409-dd5e1a9f0d94,po/2024-04-15--08-39-40--gen2-cfd079ef-3fa1-4701-9a04-3fa69a9a46e8,brizo/2020-12-22--11-33-37--session_2020_12_19_13_12_28_vm-prod-training-04_nikolay_ipace_prod_baseline_critic_seq_1_pose_seq_10_pose_loss_1,sedna/2024-07-26--14-52-16--james.turner@wayve.ai,90e9624b-d4dd-467d-a22c-2703f6c1914c/2022-05-07--11-01-53--7e509581-d561-4d31-b5a1-769852c6b1e0,poseidon/2021-04-23--10-28-15--Theepa-test,66a955ef-c523-4282-8b07-b44c09799feb/2022-12-23--11-45-56--8bbcb74f-3df9-44bf-acb5-20d06b4bb0ba,thames/2024-02-14--09-55-05--gen2-01bf197a-2ce1-435f-be68-45c342538567,16c8b7bb-f6ea-4528-aae1-697974cb547a/2021-04-07--14-55-55--b6159f20-2a95-4662-9aeb-b3f3104e588f,f685f059-498f-4db5-bee1-f872ee6a9fda/2022-06-27--11-13-58--f6eb43b2-5920-438a-86d5-23fd10099b91,1b681257-ec4a-4bd5-8159-1cc1f7ecab93/2021-03-13--16-21-20--85f503aa-7306-496d-adf9-9b6dd26652e4,neptune/2022-09-29--14-41-16--gamepad-control,ebisu/2020-02-12--20-13-46--session_2020_01_21_16_14_38_kosciuszko_przemek_480x300_baseline_exponent_0.1,neptune/2023-02-10--15-08-24--session_2022_12_05_20_46_13_host_zak_new_splits_3x,2edd5814-5a9e-417b-9493-8f4430fb9af2/2021-04-21--19-20-10--7e07ab8f-fb98-450a-a4ce-c0302029462a,48c11597-37a9-4517-8b09-b1199b0237a5/2021-12-10--08-56-33--97ae68ac-cc52-4d40-95f2-de271d60ec8e,sedna/2023-09-14--12-12-40--gamepad-control,5fb925d1-f445-4aad-a9c3-f992fcce2b3b/2021-03-03--10-48-25--3968342e-1a60-41ae-9d7a-bc33e05bc7ac,congo/2022-07-29--15-20-29--Juan,brizo/2023-04-24--14-58-44--K-Risch,ed3c46b3-a203-499e-b123-eeb7a61e6087/2022-08-27--05-54-42--002b9aa2-3a4d-41fc-ae2c-39f904a47d30,avon/2022-03-10--15-40-42--Juan,oya/2024-07-29--19-25-19--gamepad-control,2edd5814-5a9e-417b-9493-8f4430fb9af2/2022-03-11--19-05-02--cd955053-924c-44dd-abd3-96804ddeb882,neptune/2024-07-30--16-14-03--session_2024_07_29_20_02_53_host_wonjoongoo_interleaved__resourceful-pink-turtle-5__modest-purple-dalmatian-5,nereus/2023-03-09--12-26-47--session_2022_12_05_20_46_13_host_zak_new_splits_3x,884d2dba-f420-46f9-a7d6-49bb2fd6e23b/2022-02-22--10-11-41--42e41e56-29b4-40a2-9b6b-3bc4c1b16d62,66a955ef-c523-4282-8b07-b44c09799feb/2022-10-04--07-09-56--22d99c1e-d8f1-474f-9910-15bdfb274c90,dab7b7a8-a746-493d-9c20-0e69148827ef/2022-03-03--15-10-16--19dc1b56-c723-4a83-8681-db8c55fa6555,90e9624b-d4dd-467d-a22c-2703f6c1914c/2021-08-06--08-24-41--78d03a36-5191-42bf-a72e-9f3852352767,sedna/2020-09-11--09-11-34--session_2020_08_19_09_37_30_etna_nicolas_baseline_high_res_200k,oya/2023-02-06--11-22-18--session_2022_12_05_20_46_13_host_zak_new_splits_3x,16c8b7bb-f6ea-4528-aae1-697974cb547a/2021-03-13--09-54-54--c0b99262-8b85-441e-a9c3-52a8c3c1d03a,avon/2022-05-17--07-58-09--Naomi-test,brizo/2020-11-12--10-37-27--session_2020_09_28_18_50_58_vm-prod-training-dgx-01_wayve_prod_baseline_dgx_1_2M,poseidon/2024-05-17--14-23-03--gamepad-control,1b681257-ec4a-4bd5-8159-1cc1f7ecab93/2021-03-03--09-46-34--a4f4294c-be05-43d6-a7d5-ffbdbfc0259f,brizo/2024-07-22--16-10-44--session_2024_07_19_14_41_06_host_remitachet_interleaved__violet-mollusk-murmuring-5__tranquil-squid-silver-5,brizo/2022-02-14--13-23-51--gamepad-control,brizo/2021-07-16--08-09-25--Aysen-test,sedna/2024-07-06--14-16-51--session_2024_06_16_19_04_12_host_chrislinegar_epoch=0-step=0200000,moose/2024-05-16--15-36-28--gen2-f1948de4-90b9-4fb2-83ee-72d534926e4d,ebisu/2024-07-13--17-01-59--session_2024_07_12_17_49_37_host_sohamphade_interleaved__violet-mollusk-murmuring-5__kangaroo-crimson-otherworldly-1,thames/2024-06-24--20-09-05--gen2-5b65dada-e18b-4f90-84f8-4ecd229e0a11,nereus/2021-03-22--11-50-07--session_2021_03_04_22_50_51_vm-prod-training-dgx-06_wayve_spi_amp,nereus/2024-07-17--20-09-31--joe.fairclough@wayve.ai,40301abc-9988-4269-8f4b-c98d98f6cf15/2022-08-08--18-53-57--0c2c7129-4529-4d84-8c21-56ee8210c5dd,darling/2024-07-18--11-03-26--gen2-f2b948a1-9243-4291-9f55-fc63a31f25fa,90e9624b-d4dd-467d-a22c-2703f6c1914c/2021-02-20--10-00-43--c80ea6ce-82b3-44b7-9249-34722275512c,nereus/2021-12-06--10-57-13--session_2021_07_12_07_27_29_host_zak_wayve_bev_int_io,2edd5814-5a9e-417b-9493-8f4430fb9af2/2022-03-16--15-55-27--09d79f0e-c74d-4c0b-9328-7d23ba62f31f,sedna/2024-07-15--16-01-26--gamepad-control,neptune/2022-06-08--07-58-36--Naomi-test,brizo/2023-08-03--14-03-04--Kieran,neptune/2022-12-06--13-12-39--Darren-test,sedna/2023-04-21--10-48-47--session_2023_03_01_01_12_18_host_zak_unbiased-curvature,nereus/2023-01-03--10-26-25--Naomi-test,ebisu/2024-06-16--09-47-24--orlando.gomes@wayve.ai,oya/2023-09-01--13-16-44--session_2023_08_11_09_37_11_aml-a100m80gb-2nn-5_becky_si_perceiver_baseline_C3T3_pretrained_no_CA,nereus/2023-07-26--12-59-22--session_2023_07_20_06_40_57_host_zak_armadillo_1cam_no-speed-aug,sedna/2020-03-17--10-11-52--Alan,e9d917ce-d3bc-45f4-8e08-9641936fceee/2022-03-29--12-38-04--87538937-407d-40f1-aeb5-de92395e6291,e3969b44-315b-45b0-9038-a229c2a95e79/2021-02-18--13-04-37--073dfd5b-aa1d-4a7d-ab72-047b413c57b5,66a955ef-c523-4282-8b07-b44c09799feb/2022-12-18--13-31-41--d5819925-e0d6-4c59-8d77-87ee58cc0946,neptune/2024-06-01--12-33-08--session_2024_05_24_13_18_25_si_mcv_large_C3T4_arboreal_chinchilla_400k_highway_T1,16c8b7bb-f6ea-4528-aae1-697974cb547a/2021-03-14--10-33-13--30a92d90-c61c-4729-b5b6-286dd1a28df8,nammu/2020-07-13--14-57-11--session_2020_07_10_21_21_49_etna_shreyash_parametric_loss__filter_and_smooth_augment__resample__stationary_filter_30s_poly_decay__lr_0.0005,e9d917ce-d3bc-45f4-8e08-9641936fceee/2021-07-29--10-37-16--fd3e64ae-0a28-4999-9ac3-3229410f7a1b,oya/2023-05-03--15-02-22--session_2023_04_25_15_47_04_host_mike_epoch=0-step=50000,oya/2023-04-12--16-12-05--K-Risch,poseidon/2022-10-11--11-20-55--session_2022_06_09_12_49_14_aml-v100-2nn-2_tomnewton_full_databricks_data_pipeline,ebisu/2020-11-04--09-31-45--Carole--Camera-Test,19606030-f2e2-4566-9c23-a90403f617ea/2021-04-24--08-04-54--c78f9f41-5223-439e-adf0-06a292cab507,sedna/2023-03-21--14-25-26--gamepad-control,danube/2024-06-17--07-53-54--gen2-3481aedd-a803-41c5-a2b2-b599d277288a,nereus/2023-05-30--10-06-35--session_2023_04_30_20_23_35_host_zak_perceiver_heuristic_3cam_fix,66a955ef-c523-4282-8b07-b44c09799feb/2022-05-06--13-37-49--b543da54-058f-40d0-8ea6-eeb10a5508d3,nammu/2023-06-14--13-25-22--session_2023_06_07_15_12_09_aml-a100m80gb-1nn_sasha_si_perceiver_XS_C1T2_past_frame_speed_augs_no_skip,19606030-f2e2-4566-9c23-a90403f617ea/2021-09-06--07-48-06--3bddda21-1d3a-4f10-82a7-49bf0b0136c8,ebisu/2023-07-10--10-05-52--Khaled-test,e3969b44-315b-45b0-9038-a229c2a95e79/2021-11-20--15-15-20--776c3ca2-ea43-4c21-adcc-9dee0fde5a35,sedna/2021-10-20--08-48-47--Shirley-test,a026b3e4-49d6-43ec-99fa-f5a32fcbc4f5/2022-09-26--08-16-43--3b5951d1-40f8-4fef-bb29-b59b0a71cbec,brizo/2021-10-18--11-02-34--session_2021_10_14_10_55_28_dgx-v100-a_becky_mixed-camera-data-new-0.3,df937fab-f245-4852-afb7-5b8322fc09ed/2022-04-04--13-54-35--9ff74946-1fef-4ab6-9c2e-08e62b655830,b19caff4-b7f6-4d04-aec7-ee957a857b0b/2021-11-30--09-44-20--6cad54fb-9f9d-4959-af35-729931642504,ebisu/2023-06-14--09-41-13--Joe-R-test,ebisu/2023-07-13--09-07-05--session_2023_04_30_20_23_35_host_zak_perceiver_heuristic_3cam_fix,112eb825-13e5-4492-a200-6abf20b3296d/2021-11-27--10-36-57--cdd7b0e2-f74d-4287-ae88-10fe614ca40f,ebisu/2024-01-09--15-46-05--gamepad-control,nereus/2023-08-18--09-14-29--session_2023_07_31_05_22_02_host_zak_perceiver_heuristic_3time_3cam_waypoint-delta
"""

RUN_IDS = [run_id.strip() for run_id in RUN_ID_STR.split(",")]

STEP = "targetless_extrinsics"

In [ ]:
if STEP == "targetless_extrinsics":
    MAIN_CLOUD_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/extract-2024-08-01T23_00_57.681Z_cloud_main.csv"
    MAIN_LOCAL_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/run_local_main.log"
    BRANCH_CLOUD_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/extract-2024-08-01T23_03_18.981Z_cloud_branch.csv"
    BRANCH_LOCAL_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/run_local_branch.log"

if STEP == "wayveslam_visual_odometry":
    MAIN_CLOUD_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/extract-2024-08-01T23_00_57.681Z_cloud_main.csv"
    MAIN_LOCAL_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/run_local_main.log"
    BRANCH_CLOUD_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/extract-2024-08-01T23_03_18.981Z_cloud_branch.csv"
    BRANCH_LOCAL_LOGS = "/mnt/remote/data/benjin/tmp_share/notebooks/step_testing/targetless_intrinsics_out/run_local_branch.log"

In [59]:
import re

def remove_ansi_escape_codes(text):
    ansi_escape = re.compile(r'\x1b\[([0-9;]*[mK])')
    return ansi_escape.sub('', text)

def clean_file(input_file, output_file):
    # Read the content of the file
    with open(input_file, 'r') as file:
        content = file.read()

    # Remove ANSI escape codes
    cleaned_content = remove_ansi_escape_codes(content)

    # Write the cleaned content back to a new file
    with open(output_file, 'w') as file:
        file.write(cleaned_content)

# Clean the file
clean_file(BRANCH_LOCAL_LOGS, BRANCH_LOCAL_LOGS)
clean_file(MAIN_LOCAL_LOGS, MAIN_LOCAL_LOGS)

In [60]:
def _extract_run_id_and_skip_reason(message):
    run_id = message.split("run_id=")[1].split(",")[0]
    skip_reason = message.split("skip_reason=")[1]
    return run_id, skip_reason

def extract_skip_reasons_for_the_step_tested_on_cloud(logs_path: str):
    logs_path = Path(logs_path)
    df = pd.read_csv(logs_path)
    
    run_id_to_skip_reason = {}
    for _, message in df["Message"].items():
        if "mark_run_as_skipped" not in message:
            continue

        run_id, skip_reason = _extract_run_id_and_skip_reason(message)
        run_id_to_skip_reason[run_id] = skip_reason
    
    return run_id_to_skip_reason

def extract_skip_reasons_for_the_step_tested_locally(logs_path: str):
    logs_path = Path(logs_path)
    with open(logs_path, "r") as f:
        lines = f.readlines()
    
    run_id_to_skip_reason = {}
    for line in lines:
        line = line.rstrip()
        if "mark_run_as_skipped" not in line:
            continue

        run_id, skip_reason = _extract_run_id_and_skip_reason(line)
        run_id_to_skip_reason[run_id] = skip_reason
    
    return run_id_to_skip_reason


def build_dataframe_from_skip_reasons_dict(run_id_to_skip_reason: dict):
    df = pd.DataFrame(run_id_to_skip_reason.items(), columns=["run_id", "skip_reason"])
    return df

def build_output_df():
    run_id_to_skip_reason_main_cloud = extract_skip_reasons_for_the_step_tested_on_cloud(MAIN_CLOUD_LOGS)
    run_id_to_skip_reason_main_local = extract_skip_reasons_for_the_step_tested_locally(MAIN_LOCAL_LOGS)
    run_id_to_skip_reason_branch_cloud = extract_skip_reasons_for_the_step_tested_on_cloud(BRANCH_CLOUD_LOGS)
    run_id_to_skip_reason_branch_local = extract_skip_reasons_for_the_step_tested_locally(BRANCH_LOCAL_LOGS)

    df_main_cloud = build_dataframe_from_skip_reasons_dict(run_id_to_skip_reason_main_cloud)
    df_main_local = build_dataframe_from_skip_reasons_dict(run_id_to_skip_reason_main_local)
    df_branch_cloud = build_dataframe_from_skip_reasons_dict(run_id_to_skip_reason_branch_cloud)
    df_branch_local = build_dataframe_from_skip_reasons_dict(run_id_to_skip_reason_branch_local)

    df_main_merged = df_main_local.merge(df_main_cloud, on="run_id", how="outer", suffixes=("_local", "_cloud"))
    df_cloud_merged = df_branch_local.merge(df_branch_cloud, on="run_id", how="outer", suffixes=("_local", "_cloud"))
    df_merged = df_main_merged.merge(df_cloud_merged, on="run_id", how="outer", suffixes=("_main", "_branch"))

    return df_merged

skip_reasons_df = build_output_df()


In [65]:
pd.set_option('display.max_colwidth', None)  # Display full column width
skip_reasons_df

,run_id,skip_reason_local_main,skip_reason_cloud_main,skip_reason_local_branch,skip_reason_cloud_branch
0,neptune/2024-04-12--17-59-39--joe.fairclough@wayve.ai-test,test run,test run,test run,test run
1,oya/2024-06-03--06-13-30--gamepad-control,gamepad run,gamepad run,gamepad run,gamepad run
2,ed3c46b3-a203-499e-b123-eeb7a61e6087/2022-04-29--14-31-31--01c07e4e-ce73-4924-85f7-ad88e2f2eb65,Prior to intrinsic deployment,Prior to intrinsic deployment,Incomplete intrinsics,Incomplete intrinsics
3,brizo/2021-03-15--14-55-05--session_2021_01_29_17_38_26_vm-prod-training-dgx-09_wayve_fleet_baseline_route_windowing,Prior to intrinsic deployment,Prior to intrinsic deployment,Incomplete intrinsics,Incomplete intrinsics
4,dab7b7a8-a746-493d-9c20-0e69148827ef/2021-08-13--09-12-30--34e11ac1-a25b-4542-b4a7-b24d3cdc2c6b,Prior to intrinsic deployment,Prior to intrinsic deployment,Incomplete intrinsics,Incomplete intrinsics
5,brizo/2021-03-10--10-51-36--session_2021_01_29_14_10_21_vm-prod-training-04_hannes_fleet_keypoint_unet_2m_bs512_lr0.005,Prior to intrinsic deployment,Prior to intrinsic deployment,Incomplete intrinsics,Incomplete intrinsics
6,f685f059-498f-4db5-bee1-f872ee6a9fda/2021-10-20--10-01-51--2d58f2f1-5080-45f4-b7c9-6a39da78738f,Prior to intrinsic deployment,Prior to intrinsic deployment,Incomplete intrinsics,Incomplete intrinsics
7,ed3c46b3-a203-499e-b123-eeb7a61e6087/2022-05-22--17-59-44--75e19c64-5d5e-4fcc-bfcf-7c5d3d3a002b,Prior to intrinsic deployment,Prior to intrinsic deployment,Incomplete intrinsics,Incomplete intrinsics
8,nammu/2020-07-13--14-57-11--session_2020_07_10_21_21_49_etna_shreyash_parametric_loss__filter_and_smooth_augment__resample__stationary_filter_30s_poly_decay__lr_0.0005,Prior to intrinsic deployment,Prior to intrinsic deployment,Run too short for vo and targetless calibration,Run too short for vo and targetless calibration
9,73885485-1f18-45a7-bf2a-5ff5ff65beb1/2022-09-26--14-34-43--30b8febf-de2c-404a-83a6-386f5db22d7f,Prior to intrinsic deployment,Prior to intrinsic deployment,Incomplete intrinsics,Incomplete intrinsics


In [62]:
skip_run_ids = set(skip_reasons_df["run_id"])
all_run_ids = set(RUN_IDS)

assert skip_run_ids.issubset(all_run_ids), "Run ids skipped that were not used for testing, whaaaaaa...?"

succeeded_run_ids = all_run_ids - skip_run_ids

succeeded_run_ids

set()